In [ ]:
pip install pandas

In [12]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import schedule
import time

class EmailScheduler:
    def __init__(self, smtp_server, smtp_port, email_address, password):
        self.smtp_server = smtp_server
        self.smtp_port = smtp_port
        self.email_address = email_address
        self.password = password

        # Email subjects for each email
        self.subjects = [
            'Initial Email Subject',
            'Follow-up Email Subject - 3 Days Later',
            'Final Follow-up Email Subject - 5 Days Later'
        ]

    def send_emails(self, csv_file, email_index):
        """Sends emails with content based on email_index (0: initial, 1: 3-day follow-up, 2: 5-day follow-up)."""
        contacts = pd.read_csv(csv_file)

        # Set up the server with SSL
        server = smtplib.SMTP_SSL(self.smtp_server, self.smtp_port)
        server.login(self.email_address, self.password)

        for index, row in contacts.iterrows():
            recipient_email = row['email']
            company_name = row['company']

            # Email content based on email index
            email_content = f"Dear {company_name} team,\n\n"
            if email_index == 0:
                email_content += "We are reaching out to share some important updates...\n\nBest regards,\nYour Company"
            elif email_index == 1:
                email_content += "Just following up on our previous email. We'd love to hear your thoughts...\n\nBest regards,\nYour Company"
            elif email_index == 2:
                email_content += "This is our final follow-up. Please let us know if you have any questions...\n\nBest regards,\nYour Company"

            # Email message setup
            message = MIMEMultipart()
            message['From'] = f"ACM DJS <{self.email_address}>"
            message['To'] = recipient_email
            message['Subject'] = self.subjects[email_index]
            message.attach(MIMEText(email_content, 'plain'))

            try:
                # Send email
                server.sendmail(self.email_address, recipient_email, message.as_string())
                print(f"Email sent to {recipient_email}")
            except Exception as e:
                print(f"Failed to send email to {recipient_email}: {e}")

        # Close the server connection
        server.quit()

    def schedule_emails(self, csv_file):
        """Schedules the emails to be sent."""
        self.send_emails(csv_file, 0)  # Send initial email
        schedule.every(3).days.do(self.send_emails, csv_file, 1)  # Schedule 3-day follow-up
        schedule.every(5).days.do(self.send_emails, csv_file, 2)  # Schedule 5-day follow-up

    def run(self, csv_file):
        """Runs the scheduling."""
        self.schedule_emails(csv_file)
        while True:
            schedule.run_pending()
            time.sleep(86400)  # Check daily for scheduled tasks

# Example usage:
if __name__ == "__main__":
    # Email account credentials (Hostinger)
    SMTP_SERVER = 'smtp.hostinger.com'
    SMTP_PORT = 465  # SSL Port for Hostinger
    EMAIL_ADDRESS = "core@djsceacm.com"  # Only the email address
    PASSWORD = "#ACM@core2024"

    # Create an instance of EmailScheduler
    email_scheduler = EmailScheduler(SMTP_SERVER, SMTP_PORT, EMAIL_ADDRESS, PASSWORD)

    # Run the email scheduler with the specified CSV file
    email_scheduler.run('contacts.csv')


Email sent to vedantk065@gmail.com
Email sent to sharmaharsheel@gmail.com
